In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import logging
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import ourfunctions

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
X = pd.read_csv('data/Training-set-values.csv')
y = pd.read_csv('data/Training-set-labels.csv')

X['date_recorded'] = pd.to_datetime(X['date_recorded']).astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 829941045)

In [3]:
X.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [4]:

model_run = ourfunctions.Modeler(X=X, y=y)

In [5]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

numeric_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='median'))]
)

knn_preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, num_cols),
    ]
)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

knn = {'classifier': KNeighborsClassifier(), 'preprocessor': knn_preprocessor}

model_run.add_model('KNN', knn)

In [7]:
model_run.train_all()

TypeError: '<' not supported between instances of 'str' and 'int'